# O projeto é uma análise de dados que deve conter as seguintes etapas:

•	Carregar e ler arquivos .csv, xlsx ou de um banco de dados;

•	Avaliar a necessidade da transformação da variável em outra escala (agrupar idade por faixas, por exemplo);

•	Realizar a codificação das variáveis categóricas conforme os valores das variáveis (label encoder, one hot encoder ou target encoder);

•	Normalizar as variáveis;

•	Avaliar a necessidade de realizar o balanceamento da variável alvo;

•	Tratar variáveis com alta correlação;

•	Realizar a seleção de variáveis;

•	Aplicar um modelo de regressão ou classificação utilizando uma técnica de hiperparametrização automática;

•	Aplicar uma medida de avaliação do modelo.

•	Gerar uma visualização para o resultado conforme os tipos abaixo:
    - Agrupamento: gerar a visualização Scatter;
    - Classificação: gerar a matriz de confusão; 
    - Regressão: gerar a visualização da linha de saída.


In [29]:
import pandas as pd # O 'as' é um alias, um apelido. Ao invés de escrever 'pandas', escrevemos 'pd'
import numpy as np

# 1. Carregar o arquivo .csv usando o pd.read_csv()

In [30]:
df = pd.read_csv('Pokemon.csv') # Lê o arquivo .csv e armazena em um DataFrame
df.head() #  Traz os 5 primeiros registros do DataFrame

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


# 2. Avaliar a necessidade da transformação da variável em outra escala (agrupar idade por faixas, por exemplo)

Não vimos necessidade de escalar as variáveis, pois o dataset é pequeno e autoexplicativo.

# 3. Realizar a codificação das variáveis categóricas de acordo com os valores das variáveis (label encoder, one hot encoder ou target encoder)